In [ ]:
# | default_exp utils.upload_data

In [ ]:
# | exporti
import httpx
import pandas as pd
import asyncio

import domolibrary.client.Logger as lc
import domolibrary.client.DomoAuth as dmda
import domolibrary.classes.DomoDataset as dmds


In [ ]:
# | exporti
async def loop_upload(
    upload_df: pd.DataFrame,
    consol_ds: dmds.DomoDataset,
    partition_key: str,
    upload_method: str,
    logger: lc.Logger,
    debug_api: bool = False,
    debug_prn: bool = False,
    debug_fn: bool = True,
    max_retry: int = 2,
    is_index: bool = False
):
    base_msg = f"{partition_key} in {consol_ds.auth.domo_instance}" if partition_key else f"in {consol_ds.auth.domo_instance}"

    if debug_fn:
        print(
            f"starting upload of {len(upload_df)} rows to {base_msg} with {max_retry} attempts")

    retry_attempt = 0
    
    res = None

    while retry_attempt <= max_retry:
        try:
            retry_attempt += 1

            if debug_fn:
                print(f"attempt {retry_attempt} for {base_msg}")

            res = await consol_ds.upload_data(
                upload_df=upload_df,
                upload_method="REPLACE" if partition_key else upload_method,
                partition_key=partition_key,
                is_index=is_index,
                debug_api=debug_api,
                debug_prn=debug_prn,
            )

        except Exception as e:
            message = f"⚠️ upload_data : unexpected error: {e} in {partition_key} during retry_attempt {retry_attempt}"

            logger.log_warning(message)
            if debug_fn :
                print(message)
    
    return res

In [ ]:
# | export

async def upload_data(
    # instance where the data_fn function will execute against
    data_fn,  # data function to execute
    instance_auth: dmda.DomoAuth,  # instance to run the data function against
    consol_ds: dmds.DomoDataset,  # dataset where data should be accumulated
    # if partition key supplied, will replace existing partition
    partition_key: str = None,
    upload_method: str = 'REPLACE',
    is_index: bool = False,  # index dataset
    debug_prn: bool = False,
    debug_fn: bool = True,
    debug_api: bool = False,
    logger: lc.Logger = None,
    max_retry: int = 2  # number of times to attempt upload
):
    logger = logger or lc.Logger(app_name="upload_data")

    try:
        message = f"🏁 starting {instance_auth.domo_instance} - {data_fn.__name__}"
        logger.log_info(message)
        print(message)
        
        instance_session = httpx.AsyncClient()

        upload_df = await data_fn(instance_auth, instance_session, debug_api=debug_api)

        if upload_df is None or len(upload_df.index) == 0:
            message = f"no data to upload for {partition_key}: {consol_ds.id} in {consol_ds.auth.domo_instance}"
            logger.log_info(message)
            print(message)
            return None

        res = await loop_upload(
            upload_df=upload_df,
            consol_ds=consol_ds,
            partition_key=partition_key,
            upload_method=upload_method,
            debug_api=debug_api,
            debug_prn=debug_prn,
            debug_fn=debug_fn,
            max_retry=max_retry,
            logger=logger,
            is_index=False
        )

        if res.is_success:
            message = f"🚀 success upload of {partition_key} to {consol_ds.id} in {consol_ds.auth.domo_instance} in {data_fn.__name__}"
            logger.log_info(message)

        else:
            message = f"💣 upload_data successful status but failed to upload {partition_key} - {res.status} - {res.response} in {data_fn.__name__}"
            logger.log_error(message)
        
        print(message)
        
        return res

    finally:
        if is_index:

            res = await consol_ds.index_dataset(debug_api=debug_api, session=instance_session)
            if res.is_success:
                message = f"🥫 successfully indexed {consol_ds.name} in {consol_ds.auth.domo_instance}"
                logger.log_info(message)
            else:
                message = f"💀⚠️ failure to index {consol_ds.name} in {consol_ds.auth.domo_instance}"
                loger.log_error(message)
            
            print(message)

        await instance_session.aclose()


#### sample implementation of upload_data with loop

In [ ]:
import os
import domolibrary.classes.DomoBootstrap as dmbsr
import httpx
import pandas as pd


async def data_fn(
    instance_auth: dmda.DomoFullAuth,  # this API requires full auth
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
) -> pd.DataFrame:

    """function to call.  must return a dataframe."""
    try:
        bsr = dmbsr.DomoBootstrap(auth=instance_auth)
        instance_features = await bsr.get_features(debug_api=debug_api, session=session)

        upload_df = pd.DataFrame(instance_features)
        upload_df["instance"] = instance_auth.domo_instance

        return upload_df

    except Exception as e:
        print(f"getting data : unexpected error: {e}")
        return None


### get_auth
full_auth = dmda.DomoFullAuth(
    domo_instance="domo-dojo",
    domo_username="jae@onyxreporting.com",
    domo_password=os.environ["DOJO_PASSWORD"],
)

# confirm retrieves token
assert isinstance(await full_auth.get_auth_token(), str)

ds_id = "44c5af30-ea04-49e4-9d7a-529afd223590"
ds = await dmds.DomoDataset.get_from_id(dataset_id=ds_id, auth=full_auth)

await upload_data(
    instance_auth=full_auth,  # instance where the data_fn function will execute against
    consol_ds=ds,
    partition_key=full_auth.domo_instance,
    data_fn=data_fn,
    is_index=True,
    debug_fn=True,
    debug_api = False,
    max_retry=2,
)

🏁 starting domo-dojo - data_fn
starting upload of 360 rows to domo-dojo in domo-dojo with 2 attempts
attempt 1 for domo-dojo in domo-dojo
attempt 2 for domo-dojo in domo-dojo
attempt 3 for domo-dojo in domo-dojo
🚀 success upload of domo-dojo to 44c5af30-ea04-49e4-9d7a-529afd223590 in domo-dojo in data_fn
🥫 successfully indexed demo_instance_features in domo-dojo


ResponseGetData(status=200, response={'dataSourceId': '44c5af30-ea04-49e4-9d7a-529afd223590', 'uploadId': 91, 'dataTag': 'domo-dojo', 'status': 'SUCCESS', 'size': {'rowCount': 360, 'columnCount': 7, 'numberOfBytes': 20535, 'partCount': 1}, 'indexing': {'requested': False}}, is_success=True)

In [ ]:
# | export
async def upload_data_with_date(
    instance_auth,
    consol_auth,
    data_fn,
    consol_ds,
    partition_date_col,
    partition_delimiter,
    start_date,
    end_date,
    debug: bool = False,
    debug_prn: bool = False,
):

    instance_session = httpx.AsyncClient()

    print(
        f"'🎬 upload_with_data: starting retrieval {start_date}, {end_date}, {instance_auth.domo_instance}"
    )

    upload_df = await data_fn(
        instance_auth=instance_auth,
        session=instance_session,
        start_date=start_date,
        end_date=end_date,
        debug=debug,
    )

    await instance_session.aclose()

    if not isinstance(upload_df, pd.DataFrame):
        print(f"🛑 error no data returned {instance_auth.domo_instance}")
        print(upload_df)
        return None

    if debug_prn:
        print(
            f"🧻 upload_with_data: starting upload {len(upload_df)} rows for {instance_auth.domo_instance}"
        )

    task = []

    for index, partition_set in upload_df.drop_duplicates(
        subset=[partition_date_col]
    ).iterrows():
        partition_date = partition_set[partition_date_col]

        partition_key = (
            f"{instance_auth.domo_instance}{partition_delimiter}{str(partition_date)}"
        )

        task.append(
            consol_ds.upload_data(
                upload_df=upload_df[(upload_df[partition_date_col] == partition_date)],
                upload_method="REPLACE",
                partition_key=partition_key,
                is_index=False,
                debug_api=debug_api,
                debug_prn=debug_prn,
            )
        )

    res = await asyncio.gather(*task)

    if debug_prn:
        print(
            f"🎉 upload_with_data : finished uploading {len(upload_df)} rows for {instance_auth.domo_instance}"
        )
    return res